In [1]:
library(ggplot2)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
library(ggpubr)

In [3]:
load('02.differential/Deseq2_TvsN_FC2.Rdata')

In [4]:
tumor <- readRDS(file = "./00.data/Tumor.tpm.Rdata")
normal <- readRDS(file = "./00.data/Normal.tpm.Rdata")

In [5]:
phe <- readRDS("./00.data/phe_RNA54.Rdata")

In [6]:
ls()

[1] "normal"      "phe"         "rlt_high_tn" "rlt_low_tn"  "rlt_neg_tn" 
[6] "tumor"

In [7]:
phe[colnames(tumor),'group']

[1] "negative" "negative" "negative" "negative" "negative" "negative"
 [7] "negative" "Low"      "High"     "Low"      "High"     "High"    
[13] "Low"      "Low"      "negative" "negative" "negative" "negative"
[19] "negative" "High"     "Low"      "High"     "High"     "High"    
[25] "Low"      "High"     "Low"      "High"     "High"     "High"    
[31] "negative" "negative" "negative" "negative" "Low"      "High"    
[37] "Low"      "Low"      "Low"      "High"     "Low"      "Low"     
[43] "High"     "Low"      "High"     "Low"      "High"     "High"    
[49] "High"     "negative" "Low"      "Low"      "High"     "Low"

In [14]:
tumor[c('TMEM173','MB21D1'),] %>% t() %>% as.data.frame() %>% mutate(group=phe[colnames(tumor),'group'],type='tumor') -> tumor_temp
normal[c('TMEM173','MB21D1'),] %>% t() %>% as.data.frame() %>% mutate(group=phe[colnames(normal),'group'],type='normal') -> normal_temp


In [15]:
data_temp <- rbind(tumor_temp,normal_temp)

In [16]:
head(data_temp)

,TMEM173,MB21D1,group,type
,<dbl>,<dbl>,<chr>,<chr>
9DW0030,53.973213,8.728100,negative,tumor
9DW0038,28.147148,1.478817,negative,tumor
9DW0039,45.015609,6.807999,negative,tumor
9DW0049,7.601627,3.618360,negative,tumor
9DW0052,72.415940,4.373038,negative,tumor
9DW0056,71.921451,12.300306,negative,tumor


In [20]:
rlt_high_tn$all[c('TMEM173','MB21D1'),]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,category
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
TMEM173,1777.583,0.1695276,0.2289496,0.7404584,0.4590219231,0.612816751,tumor
MB21D1,204.158,0.9111747,0.2663429,3.4210595,0.0006237768,0.004085523,tumor


In [21]:
rlt_low_tn$all[c('TMEM173','MB21D1'),]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,category
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
TMEM173,2087.7864,0.8361954,0.2616871,3.195401,1.396365e-03,0.0071028381,tumor
MB21D1,224.3893,1.0612287,0.2621837,4.047653,5.173379e-05,0.0004706934,tumor


In [22]:
rlt_neg_tn$all[c('TMEM173','MB21D1'),]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,category
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
TMEM173,1874.9512,1.0921378,0.2447674,4.461942,8.122034e-06,0.0001474365,tumor
MB21D1,238.8469,0.9029909,0.2544643,3.548596,3.872909e-04,0.0030271043,tumor


In [33]:
options(repr.plot.height=4,repr.plot.width=8)
f_labels <- data.frame(group = c("High", "Low", "negative"), label = c("", "", "***\nFC>2"))
p_tmem173 <- ggboxplot(data_temp,x="type",y="TMEM173",add="jitter",add.params=list(shape=21, fill="orange", size=3)
         )+ylab("Proportion") + 
        geom_signif(comparisons=list(c("tumor", "normal")),annotations=c(""))+ 
                facet_wrap(~group, scale="free")+
     geom_text(x = 1.5, y = 110, aes(label = label), data = f_labels)

In [34]:
options(repr.plot.height=4,repr.plot.width=8)
f_labels <- data.frame(group = c("High", "Low", "negative"), label = c("", "**\nFC>2", ""))
p_mb21d1 <- ggboxplot(data_temp,x="type",y="MB21D1",add="jitter",add.params=list(shape=21, fill="orange", size=3)
         )+ylab("Proportion") + 
        geom_signif(comparisons=list(c("tumor", "normal")),annotations=c(""))+ 
                facet_wrap(~group, scale="free")+  
    geom_text(x = 1.5, y = 14.3, aes(label = label), data = f_labels)

In [100]:
ggsave(filename = "TMEM_box.pdf",plot =p_tmem173 ,width =8 ,height =4 )

In [101]:
ggsave(filename = "MB21D1_box.pdf",plot =p_mb21d1 ,width =8 ,height =4 )

### for inflammationgenes

In [42]:
inflammation_genes <- c("PSMB10", "HLA-DQA1", "HLA-DRB1","CMKLR1", "HLA-E", "NKG7", "CD8A", "CCL5", "CXCL9",
                        "CD27", "CXCR6", "IDO1","STAT1", "TIGIT", "LAG3", "CD274", "PDCD1LG2", "CD276","CCL5")

In [126]:
inflammation_genes <- unique(inflammation_genes)

In [127]:
phe %>% filter(group=="High") %>% rownames() %>% tumor[,.] -> high_tumor
phe %>% filter(group=="Low") %>% rownames() %>% tumor[,.] -> low_tumor
phe %>% filter(group=="negative") %>% rownames() %>% tumor[,.] -> neg_tumor

In [128]:
phe %>% filter(group=="High") %>% rownames() %>% normal[,.] -> high_normal
phe %>% filter(group=="Low") %>% rownames() %>% normal[,.] -> low_normal
phe %>% filter(group=="negative") %>% rownames() %>% normal[,.] -> neg_normal

In [129]:
high_tumor[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='tumor') -> hightumor_temp2
high_normal[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='normal') -> highnormal_temp2
low_tumor[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='tumor') -> lowtumor_temp2
low_normal[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='normal') -> lownormal_temp2
neg_tumor[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='tumor') -> negtumor_temp2
neg_normal[inflammation_genes,] %>% t() %>% as.data.frame() %>% mutate(type='normal') -> negnormal_temp2

In [130]:
pivot_longer(hightumor_temp2,!type, names_to = "gene", values_to = "tpm") -> hightumor_temp2
pivot_longer(lowtumor_temp2,!type, names_to = "gene", values_to = "tpm") -> lowtumor_temp2
pivot_longer(negtumor_temp2,!type, names_to = "gene", values_to = "tpm") -> negtumor_temp2
pivot_longer(highnormal_temp2,!type, names_to = "gene", values_to = "tpm") -> highnormal_temp2
pivot_longer(lownormal_temp2,!type, names_to = "gene", values_to = "tpm") -> lownormal_temp2
pivot_longer(negnormal_temp2,!type, names_to = "gene", values_to = "tpm") -> negnormal_temp2

In [131]:
rbind(hightumor_temp2,highnormal_temp2) -> High_tn
rbind(lowtumor_temp2,lownormal_temp2) -> Low_tn
rbind(negtumor_temp2,negnormal_temp2) -> Neg_tn

In [132]:
head(High_tn)

type,gene,tpm
<chr>,<chr>,<dbl>
tumor,PSMB10,155.969181
tumor,HLA-DQA1,282.394408
tumor,HLA-DRB1,856.941785
tumor,CMKLR1,5.999578
tumor,HLA-E,995.996347
tumor,NKG7,37.073469


In [133]:
#options(repr.plot.height=16,repr.plot.width=16)
p_high <- ggboxplot(High_tn,x="type",y="tpm",add="jitter",add.params=list(shape=21, fill="orange", size=3)
         )+ylab("Proportion") + facet_wrap(~gene, scale="free")+geom_signif(comparisons=list(c("tumor", "normal")),annotations=c(""))

In [134]:
ggsave(filename = "High_inflammation_box.pdf",plot =p_high ,width =10 ,height =10)

In [135]:
#options(repr.plot.height=16,repr.plot.width=16)
p_low <- ggboxplot(Low_tn,x="type",y="tpm",add="jitter",add.params=list(shape=21, fill="orange", size=3)
         )+ylab("Proportion") + facet_wrap(~gene, scale="free")+geom_signif(comparisons=list(c("tumor", "normal")),annotations=c(""))

In [136]:
ggsave(filename = "Low_inflammation_box.pdf",plot = p_low ,width =10 ,height =10)

In [137]:
#options(repr.plot.height=16,repr.plot.width=16)
p_neg <- ggboxplot(Neg_tn,x="type",y="tpm",add="jitter",add.params=list(shape=21, fill="orange", size=3)
         )+ylab("Proportion") + facet_wrap(~gene, scale="free")+   geom_signif(comparisons=list(c("tumor", "normal")),annotations=c(""))

In [138]:
ggsave(filename = "Neg_inflammation_box.pdf",plot = p_neg ,width =10 ,height =10)

In [ ]:
rlt_high_tn$diff[inflammation_genes,]